IMPORTS

In [4]:
import pandas as pd
import datetime
import spacy
from sklearn.model_selection import train_test_split

GET data from CSV files

In [5]:
# Replace 'your_file.csv' with the path to your CSV file
transaction_list = 'transaction_list_v1.csv'
store_list = 'HACK.csv'

# Read the CSV file and create a DataFrame
df = pd.read_csv(transaction_list)
stores_df = pd.read_csv(store_list, encoding = "ISO-8859-1")

# Convert Date column to date-time object
df.DATE = pd.to_datetime(df.DATE)

# Now, data_frame contains your CSV data in DataFrame format
print(df)
print(stores_df)

     CUSTOMER_KEY       DATE           SHOP_NAME     PRICE
0     UDLGCAR 858 2022-10-01  FOOD LOVERS MARKET   8091.60
1     GKQQXJR 550 2022-10-01      CHECKERS HYPER    527.57
2     UDLGCAR 858 2022-10-01                DRIP   2565.34
3     HVRBLTI 309 2022-10-01          WOOLWORTHS    881.93
4     UDLGCAR 858 2022-10-02      MR PRICE SPORT  12265.74
...           ...        ...                 ...       ...
1029   HSUWFZF 75 2023-08-30      CHECKERS HYPER   2361.43
1030  GKQQXJR 550 2023-08-30          WOOLWORTHS    435.02
1031  HVRBLTI 309 2023-08-30  THE REAL REPAIR CO   3909.05
1032  UDLGCAR 858 2023-08-30            COLUMBIA  15537.25
1033   HSUWFZF 75 2023-08-30  FOOD LOVERS MARKET  13137.12

[1034 rows x 4 columns]
                               audio               Bang & Olufsen
0                              audio                       Cell C
1                              audio                     HP Store
2                              audio        Incredible Connection
3  

Some Variables

In [6]:
#SET CUTT_OFF
cut_off = pd.to_datetime('2023-06-01')

#MAX_DATE
max_date = pd.to_datetime('2023-09-30')

# Data before cut off
observed = df[df['DATE'] < cut_off]

#SET LABEL_PERIOD_DAYS
label_period_days = 30

# Data after cut off
future = df[(df['DATE'] > cut_off) & (df['DATE'] < cut_off + pd.Timedelta(label_period_days, unit='D'))]
print(future)

    CUSTOMER_KEY       DATE                    SHOP_NAME     PRICE
739  HVRBLTI 309 2023-06-02                         TOPS  12760.51
740  GKQQXJR 550 2023-06-02                         SPAR    469.83
741  AXLVCOE 763 2023-06-02             GRAND PRIX STORE   2812.56
742  UDLGCAR 858 2023-06-02                     CHECKERS  17129.00
743  AXLVCOE 763 2023-06-03  FLOUR & FAITH HOME INDUSTRY   9326.29
..           ...        ...                          ...       ...
822  JUXDKJE 784 2023-06-28            BILTONG AND BRAAI    282.44
823   OHZPHNE 67 2023-06-28                     CHECKERS    559.27
824  GKQQXJR 550 2023-06-28                        REMAX   3334.29
825  JUXDKJE 784 2023-06-29                     CHECKERS     72.51
826   HSUWFZF 75 2023-06-29           FOOD LOVERS MARKET   1589.40

[88 rows x 4 columns]


Recency

In [7]:
def customer_recency(data, cut_off, date_column, customer_id_column):

    # Copy transactions
    cut_off = df.DATE.max()
    recency = df[df.DATE < cut_off].copy()

    # Group customers by latest transaction
    recency = recency.groupby('CUSTOMER_KEY')['DATE'].max()
    recency = ((max_date - recency).dt.days).reset_index().rename(columns={'DATE':'recency'})
    
    return recency

print(customer_recency(df, cut_off, 'DATE', 'CUSTOMER_KEY'))

  CUSTOMER_KEY  recency
0  AXLVCOE 763       35
1  GKQQXJR 550       34
2   HSUWFZF 75       32
3  HVRBLTI 309       32
4  JUXDKJE 784       35
5  NSIUPHR 651       37
6   OHZPHNE 67       36
7  RAYQLJW 475       39
8  UDLGCAR 858       33
9  ZYVHLTG 995       35


Frequency

In [8]:
def customer_frequency(data, cut_off, date_column, customer_id_column, value_column, freq='M'):

    # Copy transactions
    cut_off = df.DATE.max()
    frequency = df[df.DATE < cut_off].copy()

    # Set date column as index
    frequency.set_index('DATE', inplace=True)
    frequency.index = pd.DatetimeIndex(frequency.index)

    # Group transactions by customer key and by distinct period
    # and count transactions in each period
    frequency = frequency.groupby(['CUSTOMER_KEY', pd.Grouper(freq="M", level='DATE')]).count()
    # (Optional) Only count the number of distinct periods a transaction # occurred. Else, we will be calculating total transactions in each # period instead.

    frequency['PRICE'] = 1 # Store all distinct transactions

    # Sum transactions
    frequency = frequency.groupby('CUSTOMER_KEY').sum().reset_index().rename(columns={'PRICE' : 'frequency'})
    return frequency.drop(['SHOP_NAME'], axis=1)
print(customer_frequency(df, cut_off, 'DATE', 'CUSTOMER_KEY', 'PRICE'))

  CUSTOMER_KEY  frequency
0  AXLVCOE 763         11
1  GKQQXJR 550         11
2   HSUWFZF 75         11
3  HVRBLTI 309         11
4  JUXDKJE 784          9
5  NSIUPHR 651         11
6   OHZPHNE 67         11
7  RAYQLJW 475         10
8  UDLGCAR 858         11
9  ZYVHLTG 995         11


Monetary Value

In [9]:
def customer_value(data, cut_off, date_column, customer_id_column, value_column):

    # Copy transactions
    cut_off = df.DATE.max()
    value = df[df.DATE < cut_off].copy()

    # Set date column as index
    value.set_index('DATE', inplace=True)
    value.index = pd.DatetimeIndex(value.index)

    # Get mean or total sales amount for each customer
    value = value.groupby('CUSTOMER_KEY')['PRICE'].mean().reset_index().rename(columns={'PRICE' : 'value'})

    return value
print(customer_value(df, cut_off, 'DATE', 'CUSTOMER_KEY', 'PRICE'))

  CUSTOMER_KEY         value
0  AXLVCOE 763   2679.912794
1  GKQQXJR 550   7499.385786
2   HSUWFZF 75   5913.899764
3  HVRBLTI 309   7261.294953
4  JUXDKJE 784   1413.183243
5  NSIUPHR 651   8698.317262
6   OHZPHNE 67   1110.301346
7  RAYQLJW 475   1018.598667
8  UDLGCAR 858  11174.763527
9  ZYVHLTG 995   8533.245952


Age

In [10]:
def customer_age(data, cut_off, date_column, customer_id_column):
    # Copy transactions
    cut_off = df.DATE.max()
    age = df[df.DATE < cut_off].copy()

    # Get date of first transaction
    first_purchase = age.groupby('CUSTOMER_KEY')['DATE'].min().reset_index()

    # Get number of days between cut off and first transaction
    first_purchase['age'] = (cut_off - first_purchase['DATE']).dt.days

    return first_purchase
print(customer_age(df, cut_off, 'DATE', 'CUSTOMER_KEY'))

  CUSTOMER_KEY       DATE  age
0  AXLVCOE 763 2022-10-04  330
1  GKQQXJR 550 2022-10-01  333
2   HSUWFZF 75 2022-10-03  331
3  HVRBLTI 309 2022-10-01  333
4  JUXDKJE 784 2022-10-05  329
5  NSIUPHR 651 2022-10-02  332
6   OHZPHNE 67 2022-10-03  331
7  RAYQLJW 475 2022-10-08  326
8  UDLGCAR 858 2022-10-01  333
9  ZYVHLTG 995 2022-10-03  331


Combine all

In [11]:
def customer_rfm(data, cut_off, date_column, customer_id_column, value_column, freq='M'):
    cut_off = pd.to_datetime(cut_off)
  
    # Compute Recency
    recency = customer_recency(data, cut_off, date_column, customer_id_column)

    # Compute Frequency
    frequency = customer_frequency(data, cut_off, date_column, customer_id_column, value_column, freq=freq)

    # Compute average value
    monetary_value = customer_value(data, cut_off, date_column, customer_id_column, value_column)

    # Compute age
    age = customer_age(data, cut_off, date_column, customer_id_column)

    # Merge all columns
    return recency.merge(frequency, on=customer_id_column).merge(age, on=customer_id_column).merge(monetary_value, on=customer_id_column)

In [12]:
a = customer_rfm(df, cut_off, 'DATE', 'CUSTOMER_KEY', 'PRICE')
print(a)

  CUSTOMER_KEY  recency  frequency       DATE  age         value
0  AXLVCOE 763       35         11 2022-10-04  330   2679.912794
1  GKQQXJR 550       34         11 2022-10-01  333   7499.385786
2   HSUWFZF 75       32         11 2022-10-03  331   5913.899764
3  HVRBLTI 309       32         11 2022-10-01  333   7261.294953
4  JUXDKJE 784       35          9 2022-10-05  329   1413.183243
5  NSIUPHR 651       37         11 2022-10-02  332   8698.317262
6   OHZPHNE 67       36         11 2022-10-03  331   1110.301346
7  RAYQLJW 475       39         10 2022-10-08  326   1018.598667
8  UDLGCAR 858       33         11 2022-10-01  333  11174.763527
9  ZYVHLTG 995       35         11 2022-10-03  331   8533.245952


CHURN FUTURE

In [13]:
def generate_churn_labels(future):
    future['DidBuy'] = 1
    return future[['CUSTOMER_KEY', 'DidBuy']]

In [14]:
b = generate_churn_labels(a)
print(b)

  CUSTOMER_KEY  DidBuy
0  AXLVCOE 763       1
1  GKQQXJR 550       1
2   HSUWFZF 75       1
3  HVRBLTI 309       1
4  JUXDKJE 784       1
5  NSIUPHR 651       1
6   OHZPHNE 67       1
7  RAYQLJW 475       1
8  UDLGCAR 858       1
9  ZYVHLTG 995       1


Recursive RFM

In [15]:
def recursive_rfm(data, date_col, id_col, value_col, freq='M', start_length=30, label_period_days=30):
    # Resultant list of datasets
    dset_list = []
    # Get start and end dates of dataset
    start_date = data[date_col].min() + pd.Timedelta(start_length, unit="D")
    end_date = data[date_col].max() - pd.Timedelta(label_period_days, unit="D")
    # Get dates at desired interval
    dates = pd.date_range(start=start_date, end=end_date, freq='M')
    data[date_col] = pd.to_datetime(data[date_col])
    for cut_off in dates:
        # split by observed / future
        observed = data[data[date_col] < cut_off]
        future = data[(data[date_col] > cut_off) & (data[date_col] < cut_off + pd.Timedelta(label_period_days,  unit='D'))]
        # Get relevant columns
        rfm_columns = [date_col, id_col, value_col]
        _observed = observed[rfm_columns]
        # Compute features from observed
        rfm_features = customer_rfm(_observed, cut_off, date_col, id_col, value_col)
        # Set label for everyone who bought in 'future' as 1'
        labels = generate_churn_labels(future)
        # Outer join features with labels to ensure customers 
        # not in observed are still recorded with a label of 0
        dset = rfm_features.merge(labels, on=id_col, how='outer').fillna(0)
        dset_list.append(dset)
    # Concatenate all datasets
    full_dataset = pd.concat(dset_list, axis=0)
    res = full_dataset[full_dataset.recency != 0].dropna(axis=1, how='any')
    return res

rec_df = recursive_rfm(df, 'DATE', 'CUSTOMER_KEY', 'PRICE')
print(rec_df)

/tmp/ipykernel_12818/3333728349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future['DidBuy'] = 1
/tmp/ipykernel_12818/3333728349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future['DidBuy'] = 1
/tmp/ipykernel_12818/3333728349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

    CUSTOMER_KEY  recency  frequency       DATE  age        value  DidBuy
0    AXLVCOE 763       35         11 2022-10-04  330  2679.912794     1.0
1    AXLVCOE 763       35         11 2022-10-04  330  2679.912794     1.0
2    AXLVCOE 763       35         11 2022-10-04  330  2679.912794     1.0
3    AXLVCOE 763       35         11 2022-10-04  330  2679.912794     1.0
4    GKQQXJR 550       34         11 2022-10-01  333  7499.385786     1.0
..           ...      ...        ...        ...  ...          ...     ...
105  ZYVHLTG 995       35         11 2022-10-03  331  8533.245952     1.0
106  ZYVHLTG 995       35         11 2022-10-03  331  8533.245952     1.0
107  ZYVHLTG 995       35         11 2022-10-03  331  8533.245952     1.0
108  ZYVHLTG 995       35         11 2022-10-03  331  8533.245952     1.0
109  ZYVHLTG 995       35         11 2022-10-03  331  8533.245952     1.0

[926 rows x 7 columns]


/tmp/ipykernel_12818/3333728349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future['DidBuy'] = 1


Learning Model

In [16]:
from sklearn.model_selection import train_test_split

rec_df = rec_df.sample(frac=1) # Shuffle

# Set X and y
X = rec_df[['recency', 'frequency', 'value', 'age']]
y = rec_df[['DidBuy']].values.reshape(-1)

# Set test ratio and perform train / test split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=True)

rec_df.DidBuy.value_counts()

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# Initialize and fit model on train dataset
rf = RandomForestClassifier().fit(X_train, y_train)

# Fit on over-sampled data as well
#rf_over = RandomForestRegressor().fit(X_train_over, y_train_over)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Create Dataframe and populate with predictions and actuals
# Train set
predictions = pd.DataFrame()
predictions['true'] = y_train
predictions['preds'] = rf.predict(X_train)

# Test set
predictions_test = pd.DataFrame()
predictions_test['true'] = y_test
predictions_test['preds'] = rf.predict(X_test)

# Compute error
train_acc = accuracy_score(predictions.true, predictions.preds)
test_acc = accuracy_score(predictions_test.true, predictions_test.preds)
print(f"Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print(classification_report(predictions_test.true, predictions_test.preds))

#WE can use predict_proba(X), please check

Train Acc: 0.9959, Test Acc: 1.0000
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       186

    accuracy                           1.00       186
   macro avg       1.00      1.00      1.00       186
weighted avg       1.00      1.00      1.00       186



In [34]:


# Load the spaCy NLP model
nlp = spacy.load("en_core_web_lg")

# Sample list of stores and products
#stores = ["Apple", "ABSA Bank", "Fruit And Veg"]

def all_lower(my_list):
    return [x.lower() for x in my_list]

res = []
for ele in all_lower(df['SHOP_NAME'].unique()):
    if ele.strip():
        res.append(ele)
        
stores = res
products = ['ABSA Islamic Cheque', 'Credit card', 'Cheque', 'Contractual saving', 'Estates', 'Fixed deposit', 'Financial Management', 
            'Unit Linked products', 'Loan', 'Mortgage Loan', 'Notice fixed deposit', 'Participation bonds', 'Portfolios', 
            'Saving', 'Term deposit', 'Trusts', 'Unit Trusts', 'Wills', 'ABSA Life Assurance', 'Car Short Term Insurance', 
            'ABSA Investment Management', 'Local Documentary Products', 'Broker Budget', 'Absa Life Mulpas', 'Credit card', 
            'General ledger', 'Merchant', 'Ping It', 'Commercial Property Finance', 'Absa Working Capital Solutions', 'Pension Backed Loan']

# Function to associate a store with a product using NLP
def associate_store_with_product(store, products):
    max_similarity = 0
    associated_product = ''
    for product in products:
        # Calculate similarity between store and product using spaCy similarity score
        similarity = nlp(store).similarity(nlp(product))
        print(product)
        print(similarity)
        if similarity > max_similarity:
            max_similarity = similarity
            associated_product = product
            print(associated_product)
    print (associated_product)
    return associated_product



In [36]:
# Example usage
store_to_associate = "Audi"  # Replace this with the store you want to associate
associated_product = associate_store_with_product(store_to_associate, products)

print(f"The product associated with {store_to_associate} is: {associated_product}")

ABSA Islamic Cheque
0.0363678700665364
ABSA Islamic Cheque
Credit card
0.1788700916871432
Credit card
Cheque
0.07323286001298203
Contractual saving
0.05937706171940165
Estates
0.1207387208986501
Fixed deposit
0.09268214663695745
Financial Management
0.255889893509982
Financial Management
Unit Linked products
0.2567114769990405
Unit Linked products
Loan
0.12150045720368878
Mortgage Loan
0.14484028104294022
Notice fixed deposit
0.06560691961134725
Participation bonds
0.11341519594992991
Portfolios
0.27080125996574994
Portfolios
Saving
-0.019728954868473716
Term deposit
0.05901040844777952
Trusts
0.07460970584882205
Unit Trusts
0.162082848924737
Wills
0.015863910579552915
ABSA Life Assurance
0.15138844220912598
Car Short Term Insurance
0.3365613857454878
Car Short Term Insurance
ABSA Investment Management
0.25030723826501816
Local Documentary Products
0.19846384913358558
Broker Budget
0.2738393504099854
Absa Life Mulpas
0.06767105593921531
Credit card
0.1788700916871432
General ledger
0.0